In [2]:
pip install lxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 3.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from lxml import html
from bs4 import BeautifulSoup
import requests
import time
import re
from selenium.webdriver.common.action_chains import ActionChains

In [7]:
# WORKS!!!! Scrapes the main content
def scrape_aritzia(url, wait_time=5):
    try:
        # Set Chrome options for headless mode
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36"
        chrome_options = Options()
        # chrome_options.add_argument("--headless")
        chrome_options.add_argument(f"user-agent={user_agent}")

        # Disabling images
        prefs = {"profile.managed_default_content_settings.images": 2}
        chrome_options.add_experimental_option("prefs", prefs)

        # Initialize the WebDriver with headless mode
        driver = webdriver.Chrome(options=chrome_options)
        
        # Open the webpage
        driver.get(url)

        # Extract the item title
        product_name_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.js-product-detail__product-name'))
        )
        product_name = product_name_element.text.strip().title() if product_name_element else None
        # print(product_name)

        # Wait for the specified time before clicking the interactive element
        time.sleep(wait_time)

        # Find the interactive element
        interactive_element_xpath = '//*[@id="primary"]/div[1]/div[1]/div[3]/div/ul/li[1]/a'
        interactive_element = driver.find_element(By.XPATH, interactive_element_xpath)
        interactive_element.click()

        # Wait for the loaded content to be visible
        # loaded_content_xpath = '//*[@id="pdp-panel__details"]/div/div[2]/ul'
        loaded_content_xpath = '//*[@id="pdp-panel__details"]/div'
        loaded_element = WebDriverWait(driver, wait_time).until(
            EC.visibility_of_element_located((By.XPATH, loaded_content_xpath))
        )
        
        # Once loaded, scrape the content
        dynamic_content = loaded_element.text.strip()
        # print("printed dynamic_content:", dynamic_content)

        # Extract fabric compositions
        content_regex = r"Content:\s*(.+)"
        match = re.search(content_regex, dynamic_content)
        if match:
            materials_text = match.group(1)
            # Handle semicolon: stop parsing after first semicolon if it exists
            semicolon_index = materials_text.find(';')
            if semicolon_index != -1:
                materials_text = materials_text[:semicolon_index]
            # materials_match = re.findall(r'(\d+)%\s*([^\s,;]+)', materials_text)
            materials_match = re.findall(r'(\d+)%\s*(.*?)(?:,|$)', materials_text)
        else:
            materials_match = []

        # print("printed materials_match:", materials_match)

        # Prepare dictionary to include materials
        materials_dict = {"item": product_name}
        for percentage, material in materials_match:
            clean_material = material.lower().replace("™", "").strip()
            materials_dict[clean_material] = int(percentage)

        return materials_dict
    
    except Exception as e:
        # print(f"An error occurred: {str(e)}")
        return {'item': 'No Data', 'url': url}
        
    finally:
        # Close the WebDriver
        driver.quit()

# Example usage:
# url = "https://www.aritzia.com/us/en/product/renewal-dress/117600006.html"
# url = "https://www.aritzia.com/us/en/product/flor-top/115882013.html"
# url = "https://www.aritzia.com/us/en/product/the-%2780s-comfy-denim-shirt/120515.html?dwvar_120515_color=32984"
# url = "https://www.aritzia.com/us/en/product/lodge-linen-pant/118269.html?dwvar_118269_color=21352"
# url = "https://www.aritzia.com/us/en/product/new-power-blazer/111931.html?dwvar_111931_color=6521"
# result = scrape_aritzia(url)
# if result:
#     print(result)

In [6]:
urls = [
    "https://www.aritzia.com/us/en/product/lodge-linen-pant/118269.html?dwvar_118269_color=21352",
    "https://www.aritzia.com/us/en/product/renewal-dress/117600006.html",
    "https://www.aritzia.com/us/en/product/flor-top/115882013.html",
    "https://www.aritzia.com/us/en/product/new-power-blazer/111931.html?dwvar_111931_color=6521",
    "https://www.aritzia.com/us/en/product/the-%2780s-comfy-denim-shirt/120515.html?dwvar_120515_color=32984"
]

results = []
for url in urls:
    result = scrape_aritzia(url)
    if result:
        results.append(result)
    # else:
        #results.append({'item': 'No Data', 'url': url})

# Now, results contains all the dictionaries returned by the function
print(results)

[{'item': 'Lodge Linen Pant', 'tencel lyocell': 66, 'linen': 34}, {'item': 'No Data', 'url': 'https://www.aritzia.com/us/en/product/renewal-dress/117600006.html'}, {'item': 'Flor Top', 'lenzing ecovero viscose': 62, 'polyester': 38}, {'item': 'No Data', 'url': 'https://www.aritzia.com/us/en/product/new-power-blazer/111931.html?dwvar_111931_color=6521'}, {'item': 'No Data', 'url': 'https://www.aritzia.com/us/en/product/the-%2780s-comfy-denim-shirt/120515.html?dwvar_120515_color=32984'}]


# Below is a class

In [ ]:
# CLASS
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

class AritziaScraper:
    def __init__(self):
        self.driver = self.initialize_driver()

    def initialize_driver(self):
        # Set Chrome options for headless mode
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36")
        # Disabling images
        prefs = {"profile.managed_default_content_settings.images": 2}
        chrome_options.add_experimental_option("prefs", prefs)
        driver = webdriver.Chrome(options=chrome_options)
        return driver

    def scrape(self, url, wait_time=3):
        try:
            self.driver.get(url)
            product_name_element = WebDriverWait(self.driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.js-product-detail__product-name'))
            )
            product_name = product_name_element.text.strip().title() if product_name_element else None

            # Wait and click the interactive element
            time.sleep(wait_time)

            interactive_element_xpath = '//*[@id="primary"]/div[1]/div[1]/div[3]/div/ul/li[1]/a'
            interactive_element = self.driver.find_element(By.XPATH, interactive_element_xpath)
            time.sleep(wait_time)

            interactive_element.click()

            loaded_content_xpath = '//*[@id="pdp-panel__details"]/div'
            loaded_element = WebDriverWait(self.driver, wait_time).until(
                EC.visibility_of_element_located((By.XPATH, loaded_content_xpath))
            )
            dynamic_content = loaded_element.text.strip()

            time.sleep(wait_time)

            content_regex = r"Content:\s*(.+)"
            match = re.search(content_regex, dynamic_content)
            materials_text = match.group(1) if match else ""
            semicolon_index = materials_text.find(';')
            materials_text = materials_text[:semicolon_index] if semicolon_index != -1 else materials_text
            materials_match = re.findall(r'(\d+)%\s*(.*?)(?:,|$)', materials_text)

            time.sleep(wait_time)

            materials_dict = {"item": product_name}
            for percentage, material in materials_match:
                clean_material = material.lower().replace("™", "").strip()
                materials_dict[clean_material] = int(percentage)
            time.sleep(wait_time)
            return materials_dict

        except Exception as e:
            return {'item': 'No Data', 'url': url}

    def close_driver(self):
        self.driver.quit()

# Usage
scraper = AritziaScraper()


results = []
try:
    for url in urls:
        result = scraper.scrape(url)
        results.append(result)
finally:
    scraper.close_driver()

print(results)


# Below is scraping!

In [40]:
def scrape_aritzia1(url, wait_time=5):
    try:
        # Set Chrome options for headless mode
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36"
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument(f"user-agent={user_agent}")

        # Disabling images
        prefs = {"profile.managed_default_content_settings.images": 2}
        chrome_options.add_experimental_option("prefs", prefs)

        # Initialize the WebDriver with headless mode
        driver = webdriver.Chrome(options=chrome_options)
        
        # Open the webpage
        driver.get(url)

        # Check if the URL starts with the expected prefix
        #if not driver.current_url.startswith("https://www.aritzia.com/us/en/"):
        #    return {'item': 'Unsupported URL', 'url': url}

        # Extract the item title
        product_name_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.js-product-detail__product-name'))
        )
        product_name = product_name_element.text.strip().title() if product_name_element else None

        # Wait for the specified time before clicking the interactive element
        time.sleep(wait_time)

        # Find the interactive element
        interactive_element_xpath = '//*[@id="primary"]/div[1]/div[1]/div[3]/div/ul/li[1]/a'
        interactive_element = driver.find_element(By.XPATH, interactive_element_xpath)
        interactive_element.click()

        # Wait for the loaded content to be visible
        loaded_content_xpath = '//*[@id="pdp-panel__details"]/div'
        loaded_element = WebDriverWait(driver, wait_time).until(
            EC.visibility_of_element_located((By.XPATH, loaded_content_xpath))
        )
        
        # Once loaded, scrape the content
        dynamic_content = loaded_element.text.strip()

        # Extract fabric compositions
        content_regex = r"Content:\s*(.+)"
        match = re.search(content_regex, dynamic_content)
        if match:
            materials_text = match.group(1)
            semicolon_index = materials_text.find(';')
            if semicolon_index != -1:
                materials_text = materials_text[:semicolon_index]
            materials_match = re.findall(r'(\d+)%\s*(.*?)(?:,|$)', materials_text)
        else:
            materials_match = []

        # Prepare dictionary to include materials
        materials_dict = {"item": product_name}
        for percentage, material in materials_match:
            clean_material = material.lower().replace("™", "").strip()
            materials_dict[clean_material] = int(percentage)

        return materials_dict
    
    except Exception as e:
        return {'Error': 'Error', 'error_message': "error", 'url': url}
        
    finally:
        # Close the WebDriver
        driver.quit()

# Example usage:
# url = "https://www.aritzia.com/us/en/product/renewal-dress/117600006.html"
# url = "https://www.youtube.com/watch?v=MeBU-4Xs2RU"
# url = "https://www.aritzia.com/us/en/product/flor-top/115882013.html"
# url = "https://www.aritzia.com/us/en/product/the-%2780s-comfy-denim-shirt/120515.html?dwvar_120515_color=32984"
# url = "https://www.aritzia.com/us/en/product/lodge-linen-pant/118269.html?dwvar_118269_color=21352"
# url = "https://www.aritzia.com/us/en/product/new-power-blazer/111931.html?dwvar_111931_color=6521"
# result = scrape_aritzia(url)
# if result:
#     print(result)

In [35]:
# SCRAPING CODE
def scrape_and_update(row):
    links_list = row["Links"].split('\n')
    scraped_data_combined = ""

    for link in links_list:
        print(f"Scraping data from link: {link}")
        scraped_data = scrape_aritzia1(link)
        if scraped_data:
            scraped_data_combined += str(scraped_data) + "\n"
        time.sleep(3)
        print()

    return scraped_data_combined

In [41]:
df = pd.read_csv("small_aritzia_scores.csv")
df.head()

,title,links,video_link,brand,total_ratings,material_score,goy_rating,goy_planet,goy_people,goy_animals,price_level,url
0,My top 10 favorite Aritzia purchases; what to ...,https://www.aritzia.com/en/product/the-effortl...,https://www.youtube.com/watch?v=qO60CG6sUiQ,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia
1,My top 10 favorite Aritzia purchases; what to ...,https://www.aritzia.com/en/product/contour-tu...,https://www.youtube.com/watch?v=qO60CG6sUiQ,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia
2,My top 10 favorite Aritzia purchases; what to ...,https://www.aritzia.com/default/product/new-co...,https://www.youtube.com/watch?v=qO60CG6sUiQ,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia
3,My top 10 favorite Aritzia purchases; what to ...,https://www.aritzia.com/default/product/cozy-f...,https://www.youtube.com/watch?v=qO60CG6sUiQ,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia
4,My top 10 favorite Aritzia purchases; what to ...,https://www.aritzia.com/en/product/the-super-...,https://www.youtube.com/watch?v=qO60CG6sUiQ,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia


In [43]:
df['materials'] = df['links'].apply(scrape_aritzia1)

In [44]:
df

,title,links,video_link,brand,total_ratings,material_score,goy_rating,goy_planet,goy_people,goy_animals,price_level,url,materials
0,My top 10 favorite Aritzia purchases; what to ...,https://www.aritzia.com/en/product/the-effortl...,https://www.youtube.com/watch?v=qO60CG6sUiQ,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia,"{'Error': 'Error', 'error_message': 'error', '..."
1,My top 10 favorite Aritzia purchases; what to ...,https://www.aritzia.com/en/product/contour-tu...,https://www.youtube.com/watch?v=qO60CG6sUiQ,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia,"{'Error': 'Error', 'error_message': 'error', '..."
2,My top 10 favorite Aritzia purchases; what to ...,https://www.aritzia.com/default/product/new-co...,https://www.youtube.com/watch?v=qO60CG6sUiQ,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia,"{'Error': 'Error', 'error_message': 'error', '..."
3,My top 10 favorite Aritzia purchases; what to ...,https://www.aritzia.com/default/product/cozy-f...,https://www.youtube.com/watch?v=qO60CG6sUiQ,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia,"{'Error': 'Error', 'error_message': 'error', '..."
4,My top 10 favorite Aritzia purchases; what to ...,https://www.aritzia.com/en/product/the-super-...,https://www.youtube.com/watch?v=qO60CG6sUiQ,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia,"{'Error': 'Error', 'error_message': 'error', '..."
5,My top 10 favorite Aritzia purchases; what to ...,https://www.aritzia.com/en/product/divinity-ju...,https://www.youtube.com/watch?v=qO60CG6sUiQ,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia,"{'Error': 'Error', 'error_message': 'error', '..."
6,My top 10 favorite Aritzia purchases; what to ...,https://www.aritzia.com/en/product/contour-sq...,https://www.youtube.com/watch?v=qO60CG6sUiQ,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia,"{'Error': 'Error', 'error_message': 'error', '..."
7,My top 10 favorite Aritzia purchases; what to ...,https://www.aritzia.com/en/product/peggy-swea...,https://www.youtube.com/watch?v=qO60CG6sUiQ,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia,"{'Error': 'Error', 'error_message': 'error', '..."
8,My top 10 favorite Aritzia purchases; what to ...,https://www.aritzia.com/en/product/anthem-shor...,https://www.youtube.com/watch?v=qO60CG6sUiQ,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia,"{'Error': 'Error', 'error_message': 'error', '..."
9,Aritzia *Best Sellers* Try-on Haul | April 2024,https://www.aritzia.com/us/en/product/sinch-sm...,https://www.youtube.com/watch?v=kRrPWVP3SQ4,Aritzia,2,NaN,2,2,2,2,3,https://directory.goodonyou.eco/brand/aritzia,"{'item': 'Sinch Smooth Willow T-Shirt', 'nylon..."


In [ ]:
# df['ScrapedData'] = df.apply(scrape_and_update, axis=1)

In [56]:
df.to_csv('small_aritzia_materials.csv', index=False)

In [76]:
def scrape_aritzia2(url, wait_time=5):
    try:
        # Set Chrome options for headless mode
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36"
        chrome_options = Options()
        # chrome_options.add_argument("--headless")
        chrome_options.add_argument(f"user-agent={user_agent}")

        # Disabling images
        prefs = {"profile.managed_default_content_settings.images": 2}
        chrome_options.add_experimental_option("prefs", prefs)

        # Initialize the WebDriver with headless mode
        driver = webdriver.Chrome(options=chrome_options)
        
        # Open the webpage
        driver.get(url)

        # Extract the item title
        product_name_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.js-product-detail__product-name'))
        )
        product_name = product_name_element.text.strip().title() if product_name_element else None

        # Wait for the specified time before clicking the interactive element
        time.sleep(wait_time)

        # Find the interactive element
        interactive_element_xpath = '//*[@id="primary"]/div[1]/div[1]/div[3]/div/ul/li[1]/a'
        interactive_element = driver.find_element(By.XPATH, interactive_element_xpath)
        interactive_element.click()

        # Wait for the loaded content to be visible
        loaded_content_xpath = '//*[@id="pdp-panel__details"]/div'
        loaded_element = WebDriverWait(driver, wait_time).until(
            EC.visibility_of_element_located((By.XPATH, loaded_content_xpath))
        )
        
        # Once loaded, scrape the content
        dynamic_content = loaded_element.text.strip()

        # Extract fabric compositions
        content_regex = r"Content:\s*(.+)"
        match = re.search(content_regex, dynamic_content)
        if match:
            materials_text = match.group(1)
            semicolon_index = materials_text.find(';')
            if semicolon_index != -1:
                materials_text = materials_text[:semicolon_index]
            materials_match = re.findall(r'(\d+)%\s*(.*?)(?:,|$)', materials_text)
        else:
            materials_match = []

        # Prepare dictionary to include materials
        materials_dict = {"item": product_name}
        for percentage, material in materials_match:
            clean_material = material.lower().replace("™", "").strip()
            materials_dict[clean_material] = int(percentage)

        return materials_dict
    
    except Exception as e:
        return {'Error': 'Error', 'error_message': "error", 'url': url}
        
    finally:
        # Close the WebDriver
        driver.quit()

result = scrape_aritzia2("https://www.aritzia.com/us/en/product/divinity-jumpsuit/99810.html?country=us")
if result:
    print(result)

{'item': 'Divinity Jumpsuit', 'nylon': 82, 'elastane': 18}


In [54]:
def get_material_score(item):
    # List of accepted parameters by the calculator
    material_params = ["COTTON", "RECYCLED_COTTON", "ORGANIC_COTTON", 
                    "POLYESTER", "RECYCLED_POLYESTER", "NYLON", 
                    "RECYCLED_NYLON", "ACRYLIC", "SPANDEX", 
                    "FLAX", "LINEN", "HEMP", "CUPRO", 
                    "LYOCELL", "TENCEL_LYOCELL_LENZING", 
                    "REFIBRA_TENCEL_LYOCELL_LENZING", "MODAL", 
                    "TENCEL_MODAL_LENZING", "VISCOSE", "VISCOSE_BAMBOO", 
                    "VISCOSE_ASIA_LENZING", "VISCOSE_EU_LENZING",
                    "SILK", "ALPACA", "WOOL", "RECYCLED_WOOL", 
                    "CASHMERE", "RECYCLED_CASHMERE"]
    
    # Base calculator URL
    calculator = "https://www.selflessclothes.com/blog/sustainability-calculator/?"

    # Extract the materials out of the item dictionary for calculator URL's query parameters
    materials_and_percents = {key: value for key, value in item.items() if key != 'item'}

    goal_params = []

    # Check if the materials are in the material parameters that the website uses
    for material, percent in materials_and_percents.items():
        if material.upper() in material_params:
            goal_params.extend([f"material={material.upper()}", f"percentage={percent}"])

    # Reorder the parameters so they match the score calculator
    reordered_params = [goal_params[i] for i in range(0, len(goal_params), 2)] + [goal_params[i] for i in range(1, len(goal_params), 2)]

    # String concatenation of the score calculator url
    score_calculator = calculator + "&".join(reordered_params)

    # Configure Chrome options for headless mode
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode

    # Initialize Chrome WebDriver with configured options
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # Load the score calculator page with the results
        driver.get(score_calculator)

        # Wait for the score element to be present
        score_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ml-2"))
        )

        # Extract the score text
        score_text = score_element.text.strip()

        # Extract the score from the parentheses and convert it to a float
        material_score = float(score_text[score_text.find('(') + 1: score_text.find(')')])

        # Returns the clothing item's name and material score
        return (item.get("item"), f"Material Score: {material_score}")
    except Exception as e:
        print("Error:", e)
        return None
    finally:
        # Close the WebDriver
        driver.quit()

In [95]:
result = scrape_aritzia2("https://www.aritzia.com/us/en/product/program-pant/102575.html?dwvar_102575_color=1274")
if result:
    print(result)

{'item': 'Program Pant', 'polyurethane': 100}


In [96]:
item = {'item': 'Program Pant', 'spandex': 100}


get_material_score(item)

('Program Pant', 'Material Score: 0.9')